In [35]:
import numpy as np
from keras.layers import Dense, Flatten,Activation,Dropout
from keras.applications import VGG16
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils

In [9]:
model = VGG16(weights='imagenet',include_top=False)

In [13]:
#model.summary()

In [16]:
datagen = ImageDataGenerator(
    rescale=1./255,
    #horizontal_flip=True
)
batch_size=2

In [40]:
train_generator = datagen.flow_from_directory(
        './train2/',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode=None,  # this means our generator will only yield batches of data, no labels
        shuffle=False)  

Found 80 images belonging to 2 classes.


In [20]:
# validation_generator = datagen.flow_from_directory(
#     './validation/',
#     target_size=(150,150),
#     batch_size=16,
#     class_mode=None
# )

In [39]:
bottleneck_features_train = model.predict_generator(train_generator, 80)
np.save(open('bottleneck_features_train2.npy', 'w'), bottleneck_features_train)

KeyboardInterrupt: 

In [27]:
train_data = np.load(open('./bottleneck_features_train3.npy'))
print train_data.shape

(8, 4, 4, 512)


In [31]:
topmodel = Sequential()
topmodel.add(Flatten(input_shape=train_data.shape[1:]))
topmodel.add(Dense(64))
topmodel.add(Activation('relu'))
topmodel.add(Dropout(0.25))
topmodel.add(Dense(2))
topmodel.add(Activation('softmax'))

In [33]:
train_labels = []
for ix in range(train_data.shape[0]/2):
    train_labels.append(0)
for ix in range(train_data.shape[0]/2):
    train_labels.append(1)

In [36]:
train_labels = np.asarray(train_labels)
train_labels = np_utils.to_categorical(train_labels)
print train_labels.shape

(8, 2)


In [37]:
topmodel.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [38]:
topmodel.fit(train_data,train_labels,nb_epoch=10,batch_size=2)

Epoch 1/10
8/8 [==============================] - 0s - loss: 1.4019 - acc: 0.2500         
Epoch 2/10
8/8 [==============================] - 0s - loss: 0.6824 - acc: 0.7500     
Epoch 3/10
8/8 [==============================] - 0s - loss: 0.0530 - acc: 1.0000     
Epoch 4/10
8/8 [==============================] - 0s